In [1]:
%%capture
# Capture because braindecode module prints a lot :S
from pyFiles.training import *
from pyFiles.models import *
from pyFiles.dataprep_down import *
from pyFiles.dataprep2_down import *
#
import torch
from torch import nn
from torch.nn import functional as F
#
from pyFiles.losses import InfoNceLoss, VICReg, BarlowTwins

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [3]:
#PREP DENUEVO
# prep("C:\\Users\\TheSy\\Desktop\\dataverse_files", mode = "per_win", save = True, save_dir = "data_down")

In [4]:
# spliter = DFSpliter(save = True, mode= "down_win")
# spliter("down_prep_windows.csv",".")

In [5]:
transforms = Augmentations(5, False, AUGMENTATIONS)

In [6]:
train_dataset = CustomEEGDataset("down_win_train_feats.csv", root_dir = "", transform = transforms)
val_dataset = CustomEEGDataset("down_win_val_feats.csv", root_dir = "")

In [7]:
trainloader = DataLoader(train_dataset, batch_size = 100, shuffle = True, num_workers = 0)
valloader = DataLoader(val_dataset, batch_size = 100, shuffle = False, num_workers = 0)

In [8]:
model = SleepStager(n_chans = 19, pretext = False)
# enc, fc = torch.load("multis/infonce/bestEncoderParamsmulti_inf.pt")
# model.encoder.load_state_dict(enc)

criterion = nn.CrossEntropyLoss()

#FREEZE
for param in model.encoder.parameters():
  param.requires_grad = False

optimizer = torch.optim.Adam(model.final.parameters(), lr = 0.001)
downtrain(model, 1, trainloader, valloader, criterion, optimizer, state = None, name = "multi_inf")

Epoch nro 1/1
Iter: 1/15, Loss:-0.0
Iter: 2/15, Loss:-0.0
Iter: 3/15, Loss:-0.0
Iter: 4/15, Loss:-0.0
Iter: 5/15, Loss:-0.0
Iter: 6/15, Loss:-0.0
Iter: 7/15, Loss:-0.0
Iter: 8/15, Loss:-0.0
Iter: 9/15, Loss:-0.0
Iter: 10/15, Loss:-0.0
Iter: 11/15, Loss:-0.0
Iter: 12/15, Loss:-0.0
Iter: 13/15, Loss:-0.0
Iter: 14/15, Loss:-0.0
Iter: 15/15, Loss:-0.0
Epoch 1/1: Train loss = 0.0, Val loss = 0.0, Train acc = 44.400001525878906, Val acc = 45.5
Better params found in epoch = 1, saved params


([[0.0], [0.0]],
 [[tensor(44.4000, device='cuda:0')], [tensor(45.5000, device='cuda:0')]])